In [9]:
import joblib
import gzip
import json
import numpy as np
import random
import pandas as pd
import os
from sqlalchemy import create_engine
import urllib
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from transformers import pipeline

In [10]:
## Login to Azure Key Vault

# replace 'my-key-vault-url' with the URL of your Azure Key Vault
vault_url = 'https://kv-colmex.vault.azure.net/'

# DefaultAzureCredential will use the credentials of your logged-in Azure account
credential = DefaultAzureCredential()

# create a SecretClient
client = SecretClient(vault_url=vault_url, credential=credential)

In [11]:
# Read reviews data and extract sample of 100 rows
df = pd.read_parquet("../data_files/data_industry.parquet").sample(100, random_state=5)

In [12]:
print(df['asin'].drop_duplicates().to_list())

['B000UXXUSG', 'B0002Q80TA', 'B0026PI8JC', 'B000RA5BMK', 'B00EZKO8J0', 'B015724OVG', 'B005CSF1JK', 'B00E6LJBUO', 'B00505VMXY', 'B000Q6ZK3K', 'B0091KOI8K', 'B000XHE0Q0', 'B0095CC1PQ', 'B00E6LJAOG', 'B00JTE1Z8E', 'B000W3RSGE', 'B001B5L5SY', 'B00DQG8SR2', 'B000SKZIXG', 'B000IVEM5K', 'B00E6LJ2SA', 'B004QK8FBG', 'B000F7VRIC', 'B00CFRF7UY', 'B015VYC606', 'B0009XB160', 'B000K757W4', 'B0026PEQES', 'B003PDMNCC', 'B002DHLUWK', 'B015724RQI', 'B005AAWLYS', 'B002MFSRIS', 'B013XFGA6G', 'B008S0IV9G', 'B00CTTEKJW', 'B00EZQYBZ0', 'B015IHWAZW', 'B004A7JMSU', 'B01326J80Q', 'B0014X5XEK', '3959828276', 'B012F7PNPM', 'B003DU34P2', 'B0002XHAX0', 'B001C31OZY', 'B000ES4PYU', 'B00ENFVLAQ', 'B009348X7Q', 'B001C3MHTQ', 'B00EZJRS8E', 'B015NN1E0S', 'B01637RMYU', 'B000AP2X0A', 'B001B5J2UW', 'B0144NYGJY', 'B00A6TPHZS', 'B003VNCRNQ', 'B0095C08YM', 'B004Q0PT3I', 'B00005AC56', 'B00111DJQ4', 'B00M9GTHS4', 'B000X86ZAS', 'B004YHXXKO', 'B00002SANG', 'B000BZJ0LY', 'B000HCZ8EO', 'B004A7Y0UK', 'B008MR38ZM', 'B000HLT5HQ', 'B015

In [13]:
# Select only necessary columns
df = df[['asin', 'overall', 'reviewText', 'reviewerID', 'reviewerName',
       'summary', 'unixReviewTime', 'verified', 'vote']]

In [14]:
# Create column of review ID 
df['reviewID'] = df['reviewerID'] + "-" + df['asin'] + "-" + df['unixReviewTime']

In [15]:
# Create date column based on column unixReviewTime
df['dateReview'] = pd.to_datetime(df['unixReviewTime'].astype('int'), unit='s')

In [16]:
# Load ML models for sentiment and emotion recognition on base of text
model_sentiment = "cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_task = pipeline("sentiment-analysis", model=model_sentiment, tokenizer=model_sentiment)

model_emotion = "j-hartmann/emotion-english-distilroberta-base"
emotion_task = pipeline("text-classification", model=model_emotion, return_all_scores=False)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/home/jose/miniconda3/envs/azure_download/lib/python3.11/site-packages/transformers/pipelines/text_classificati

In [17]:
# In case the review text is too long, use the summary to detect sentiment
def safe_sentiment_task(row):
    try:
        return tuple(sentiment_task(row['reviewText'])[0].values())
    except RuntimeError:
        try:
            return tuple(sentiment_task(row['summary'], **{"truncation": True, "max_length": 512})[0].values())
        except (RuntimeError, IndexError):
                return tuple('neutral', 0)

# The same for emotion
def safe_emotion_task(row):
    try:
        return tuple(emotion_task(row['reviewText'])[0].values())
    except RuntimeError:
        try:
            return tuple(emotion_task(row['summary'], **{"truncation": True, "max_length": 512})[0].values())
        except (RuntimeError, IndexError):
                return tuple('neutral', 0)

In [18]:
sentiment = df.apply(safe_sentiment_task, axis=1)
emotion = df.apply(safe_emotion_task, axis=1)

Token indices sequence length is longer than the specified maximum sequence length for this model (1300 > 512). Running this sequence through the model will result in indexing errors


In [19]:
df[['sentiment', 'sentiment_score']] = pd.DataFrame(sentiment.to_list(), index=df.index)
df[['emotion', 'emotion_score']] = pd.DataFrame(emotion.to_list(), index=df.index)

In [20]:
# Drop columns of scores
df.drop(columns=['sentiment_score', 'emotion_score'], inplace=True)

In [21]:
df.pivot_table(columns=['sentiment'], index=['emotion'], aggfunc='count', values='reviewID', margins=True)

sentiment,negative,neutral,positive,All
emotion,,,,
anger,3.0,NaN,3.0,6
disgust,5.0,1.0,NaN,6
fear,1.0,1.0,1.0,3
joy,NaN,1.0,20.0,21
neutral,13.0,9.0,27.0,49
sadness,10.0,NaN,NaN,10
surprise,1.0,NaN,4.0,5
All,33.0,12.0,55.0,100


In [22]:
df.columns

Index(['asin', 'overall', 'reviewText', 'reviewerID', 'reviewerName',
       'summary', 'unixReviewTime', 'verified', 'vote', 'reviewID',
       'dateReview', 'sentiment', 'emotion'],
      dtype='object')

In [23]:
# Drop duplicates
df.drop_duplicates(subset=['reviewID'], inplace=True)

In [24]:
# parameters
server = client.get_secret('db-server').value
database = client.get_secret('database').value
username = client.get_secret('db-username').value
password = client.get_secret('db-password').value
driver = '{ODBC Driver 17 for SQL Server}'
table = 'reviews_test'

# create the connection string
params = urllib.parse.quote_plus(
    f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}')

engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}')

df.to_sql(f'{table}', con=engine, if_exists='append', index=False)

100